In [1]:
import torch
import random
import zipfile
import time
import math
import numpy as np
from torch import nn, optim
import torch.nn.functional as F

In [2]:
def load_data_jay_lyrics():
    with zipfile.ZipFile('jaychou_lyrics.txt.zip') as zin:
        with zin.open('jaychou_lyrics.txt') as f:
            corpus_chars = f.read().decode('utf-8')
    corpus_chars = corpus_chars.replace('\n', ' ').replace('\r', ' ')
    corpus_chars = corpus_chars[0:10000]
    idx_to_char = list(set(corpus_chars))
    char_to_idx = dict([(char, i) for i, char in enumerate(idx_to_char)])
    vocab_size = len(char_to_idx)
    corpus_indices = [char_to_idx[char] for char in corpus_chars]
    sample = corpus_indices[:20]
    return corpus_indices, char_to_idx, idx_to_char, vocab_size

def data_iter_random(corpus_indices, batch_size, num_steps, device=None):
    # 减1是因为输出的索引x是相应输入的索引y加1
    num_examples = (len(corpus_indices) -1 ) // num_steps
    epoch_size = num_examples // batch_size
    example_indices = list(range(num_examples))
    random.shuffle(example_indices)
    
    def _data(pos):
        return corpus_indices[pos: pos+num_steps]
    if device is None:
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    for i in range(epoch_size):
        #每次读取batch_size个随机样本
        i = i* batch_size
        batch_indices = example_indices[i: i+batch_size]
        X = [_data(j * num_steps) for j in batch_indices]
        Y = [_data(j * num_steps+1) for j in batch_indices]
        yield torch.tensor(X, dtype=torch.float32, device=device), torch.tensor(Y, dtype=torch.float32, device=device)
    
def data_iter_consecutive(corpus_indices, batch_size, num_steps, device=None):
    if device == None:
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    corpus_indices = torch.tensor(corpus_indices, dtype=torch.float32, device=device)
    data_len = len(corpus_indices)
    batch_len = data_len // batch_size
    indices = corpus_indices[0: batch_size*batch_len].view(batch_size, batch_len)
    epoch_size = (batch_len - 1)  // num_steps
    for i in range(epoch_size):
        i = i * num_steps
        X = indices[:, i:i+num_steps]
        Y = indices[:, i+1 : i+num_steps+1]
        yield X, Y

def one_hot(x, n_class, dtype=torch.float32, device=None):
    if device == None:
        device = 'cuda' if torch.cuda.is_available() else 'cpu'
    x = x.long().to(device)
    res = torch.zeros(x.shape[0], n_class, dtype=dtype, device=device)
    res.scatter_(1, x.view(-1, 1), 1)
    return res

def to_onehot(X, n_class):
    return [one_hot(X[:,i], n_class) for i in range(X.shape[1])]

def grad_clipping(params, theta, device):
    norm = torch.tensor([0.0], device=device)
    for param in params:
        norm += (param.grad.data ** 2).sum()
    norm = norm.sqrt().item()
    if norm > theta:
        for param in params:
            param.grad.data *= (theta / norm)
            
def predict_rnn(prefix, num_chars, rnn, params, init_rnn_state,
               num_hiddens, vocab_size, device, idx_to_char, char_to_idx):
    state = init_rnn_state(1, num_hiddens, device)
    output = [char_to_idx[prefix[0]]]
    for t in range(num_chars + len(prefix) - 1):
        # 将上一时间步的输出作为当前时间步的输入
        X = to_onehot(torch.tensor([[output[-1]]], device=device), vocab_size)
        # 计算输出和更新隐藏状态
        (Y, state) = rnn(X, state, params)
        # 下一个时间步的输入是prefix里的字符或者当前的最佳预测字符
        if t < len(prefix) - 1:
            output.append(char_to_idx[prefix[t+1]])
        else:
            output.append(int(Y[0].argmax(dim=1).item()))
    return ''.join([idx_to_char[i] for i in output])

def sgd(params, lr, batch_size):
    #定义优化算法
    for param in params:
        param.data -= lr * param.grad / batch_size

# 循环神经网络

## 含隐藏状态的循环神经网络
验证$ X_tW_{xh}+H_{t-1}W_{hh}$ 的计算等价$X_tH_{t-1}$连结后的矩阵乘以$ W_{xh}$与$ W_{hh}$连结后的矩阵

In [3]:
X, W_xh = torch.randn(3, 1), torch.randn(1,4)
H, W_hh = torch.randn(3, 4), torch.rand(4,4) #H为X*W_xh
torch.mm(X, W_xh) + torch.mm(H, W_hh)

tensor([[ 1.2250,  0.9437,  1.1191,  0.0807],
        [ 0.1978,  0.0854, -1.8729,  0.0352],
        [-0.4734, -0.3462, -0.4319, -1.0958]])

In [4]:
torch.matmul(torch.cat((X, H), dim=1), torch.cat((W_xh, W_hh), dim=0))

tensor([[ 1.2250,  0.9437,  1.1191,  0.0807],
        [ 0.1978,  0.0854, -1.8729,  0.0352],
        [-0.4734, -0.3462, -0.4319, -1.0958]])

# 语言模型数据集（周杰伦专辑歌词）

In [5]:
with zipfile.ZipFile('jaychou_lyrics.txt.zip') as zin:
    with zin.open('jaychou_lyrics.txt') as f:
        corpus_chars = f.read().decode('utf-8')
corpus_chars[:40]

'想要有直升机\n想要和你飞到宇宙去\n想要和你融化在一起\n融化在宇宙里\n我每天每天每'

In [6]:
#把换行符替换成空格，然后仅使用前1万个字符来训练模型
corpus_chars = corpus_chars.replace('\n', ' ').replace('\r', ' ')
corpus_chars = corpus_chars[0:10000]

## 建立字符索引

In [7]:
idx_to_char = list(set(corpus_chars))
char_to_idx = dict([(char, i) for i, char in enumerate(idx_to_char)])
vocab_size = len(char_to_idx)
vocab_size

1027

In [8]:
#训练数据集中每个字符转化为索引，并打印前20个字符及其对应的索引
corpus_indices = [char_to_idx[char] for char in corpus_chars]
sample = corpus_indices[:20]
print('chars:', ''.join([idx_to_char[idx] for idx in sample]))
print('indices:', sample)


chars: 想要有直升机 想要和你飞到宇宙去 想要和
indices: [741, 161, 913, 701, 886, 876, 931, 741, 161, 805, 1010, 690, 840, 465, 471, 862, 931, 741, 161, 805]


## 时序数据的采样

### 随机采样
下面的代码每次从数据里随机采样一个小批量。其中批量大小batch_size指每个小批量的样本数，num_steps为每个样本所包含的时间步数。 在随机采样中，每个样本是原始序列上任意截取的一段序列。相邻的两个随机小批量在原始序列上的位置不一定相毗邻。因此，我们无法用一个小批量最终时间步的隐藏状态来初始化下一个小批量的隐藏状态。在训练模型时，每次随机采样前都需要重新初始化隐藏状态。

In [9]:
def data_iter_random(corpus_indices, batch_size, num_steps, device=None):
    # 减1是因为输出的索引x是相应输入的索引y加1
    num_examples = (len(corpus_indices) -1 ) // num_steps
    epoch_size = num_examples // batch_size
    example_indices = list(range(num_examples))
    random.shuffle(example_indices)
    
    def _data(pos):
        return corpus_indices[pos: pos+num_steps]
    if device is None:
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    for i in range(epoch_size):
        #每次读取batch_size个随机样本
        i = i* batch_size
        batch_indices = example_indices[i: i+batch_size]
        X = [_data(j * num_steps) for j in batch_indices]
        Y = [_data(j * num_steps+1) for j in batch_indices]
        yield torch.tensor(X, dtype=torch.float32, device=device), torch.tensor(Y, dtype=torch.float32, device=device)

In [10]:
my_seq = list(range(30))
for X, Y in data_iter_random(my_seq, batch_size=2, num_steps=6):
    print('X: ', X, '\nY:', Y, '\n')

X:  tensor([[12., 13., 14., 15., 16., 17.],
        [ 6.,  7.,  8.,  9., 10., 11.]], device='cuda:0') 
Y: tensor([[13., 14., 15., 16., 17., 18.],
        [ 7.,  8.,  9., 10., 11., 12.]], device='cuda:0') 

X:  tensor([[ 0.,  1.,  2.,  3.,  4.,  5.],
        [18., 19., 20., 21., 22., 23.]], device='cuda:0') 
Y: tensor([[ 1.,  2.,  3.,  4.,  5.,  6.],
        [19., 20., 21., 22., 23., 24.]], device='cuda:0') 



### 相邻采样
当多个相邻小批量通过传递隐藏状态串联起来时，模型参数的梯度计算将依赖所有串联起来的小批量序列。同一迭代周期中，随着迭代次数的增加，梯度的计算开销会越来越大。 为了使模型参数的梯度计算只依赖一次迭代读取的小批量序列，我们可以在每次读取小批量前将隐藏状态从计算图中分离出来。我们将在下一节（循环神经网络的从零开始实现）的实现中了解这种处理方式。

In [11]:
def data_iter_consecutive(corpus_indices, batch_size, num_steps, device=None):
    if device == None:
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    corpus_indices = torch.tensor(corpus_indices, dtype=torch.float32, device=device)
    data_len = len(corpus_indices)
    batch_len = data_len // batch_size
    indices = corpus_indices[0: batch_size*batch_len].view(batch_size, batch_len)
    epoch_size = (batch_len - 1)  // num_steps
    for i in range(epoch_size):
        i = i * num_steps
        X = indices[:, i:i+num_steps]
        Y = indices[:, i+1 : i+num_steps+1]
        yield X, Y

In [12]:
#同样的设置下，打印相邻采样每次读取的小批量样本的输入X和标签Y。
#相邻的两个随机小批量在原始序列上的位置相毗邻
for X, Y in data_iter_consecutive(my_seq, batch_size=2, num_steps=6):
    print('X: ', X, '\nY:', Y, '\n')

X:  tensor([[ 0.,  1.,  2.,  3.,  4.,  5.],
        [15., 16., 17., 18., 19., 20.]], device='cuda:0') 
Y: tensor([[ 1.,  2.,  3.,  4.,  5.,  6.],
        [16., 17., 18., 19., 20., 21.]], device='cuda:0') 

X:  tensor([[ 6.,  7.,  8.,  9., 10., 11.],
        [21., 22., 23., 24., 25., 26.]], device='cuda:0') 
Y: tensor([[ 7.,  8.,  9., 10., 11., 12.],
        [22., 23., 24., 25., 26., 27.]], device='cuda:0') 



# 循环神经网络从零开始实现

In [13]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

(corpus_indices, char_to_idx, idx_to_char, vocab_size) = load_data_jay_lyrics()

## one-hot向量

In [14]:
def one_hot(x, n_class, dtype=torch.float32, device=None):
    if device == None:
        device = 'cuda' if torch.cuda.is_available() else 'cpu'
    x = x.long().to(device)
    res = torch.zeros(x.shape[0], n_class, dtype=dtype, device=device)
    res.scatter_(1, x.view(-1, 1), 1)
    return res

x = torch.tensor([0, 2])
one_hot(x, vocab_size)

tensor([[1., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 1.,  ..., 0., 0., 0.]], device='cuda:0')

In [15]:
def to_onehot(X, n_class):
    return [one_hot(X[:,i], n_class) for i in range(X.shape[1])]
#感觉这里列变成了样本号，行变成了特征，转置了
X = torch.arange(10).view(2, 5)
inputs = to_onehot(X, vocab_size)
print(len(inputs), inputs[0].shape)

5 torch.Size([2, 1027])


## 初始化模型参数

In [16]:
num_inputs, num_hiddens, num_outputs = vocab_size, 256, vocab_size
print('will use', device)
def get_params():
    def _one(shape):
        ts = torch.tensor(np.random.normal(0, 0.01, size=shape), device=device, dtype=torch.float32)
        return torch.nn.Parameter(ts,  requires_grad=True)
    
    #隐藏层参数
    W_xh = _one((num_inputs, num_hiddens))
    W_hh = _one((num_hiddens, num_hiddens))
    b_h = torch.nn.Parameter(torch.zeros(num_hiddens, device=device, requires_grad=True))
    #输出层参数
    W_hq = _one((num_hiddens,num_outputs))
    b_q = torch.nn.Parameter(torch.zeros(num_outputs, device=device, requires_grad=True))
    
    return nn.ParameterList([W_xh, W_hh, b_h, W_hq, b_q])

will use cuda


## 定义模型
首先定义init_rnn_state函数来返回初始化的隐藏状态。它返回由一个形状为(批量大小, 隐藏单元个数)的值为0的NDArray组成的元组。使用元组是为了更便于处理隐藏状态含有多个NDArray的情况。

In [17]:
def init_rnn_state(batch_size, num_hiddens, device):
    return (torch.zeros((batch_size, num_hiddens), device=device), )

下面的rnn函数定义了在一个时间步里如何计算隐藏状态和输出。这里的激活函数使用了tanh函数

In [18]:
def rnn(inputs, state, params):
    W_xh, W_hh, b_h, W_hq, b_q = params
    H, = state
    outputs = []
    for X in inputs:
        H = torch.tanh(torch.mm(X, W_xh) + torch.mm(H, W_hh) + b_h)
        Y = torch.matmul(H, W_hq) + b_q
        outputs.append(Y)
    return outputs, (H,)

In [19]:
#做个简单的测试来观察输出结果的个数（时间步数），以及第一个时间步的输出层输出的形状和隐藏状态的形状。
state = init_rnn_state(X.shape[0], num_hiddens, device)
inputs = to_onehot(X.to(device), vocab_size)
params = get_params()
outputs, state_new = rnn(inputs, state, params)
print(len(outputs), outputs[0].shape, state_new[0].shape)

5 torch.Size([2, 1027]) torch.Size([2, 256])


## 定义预测函数
以下函数基于前缀prefix（含有数个字符的字符串）来预测接下来的num_chars个字符。这个函数稍显复杂，其中我们将循环神经单元rnn设置成了函数参数，这样在后面小节介绍其他循环神经网络时能重复使用这个函数。

In [20]:
def predict_rnn(prefix, num_chars, rnn, params, init_rnn_state,
               num_hiddens, vocab_size, device, idx_to_char, char_to_idx):
    state = init_rnn_state(1, num_hiddens, device)
    output = [char_to_idx[prefix[0]]]
    for t in range(num_chars + len(prefix) - 1):
        # 将上一时间步的输出作为当前时间步的输入
        X = to_onehot(torch.tensor([[output[-1]]], device=device), vocab_size)
        # 计算输出和更新隐藏状态
        (Y, state) = rnn(X, state, params)
        # 下一个时间步的输入是prefix里的字符或者当前的最佳预测字符
        if t < len(prefix) - 1:
            output.append(char_to_idx[prefix[t+1]])
        else:
            output.append(int(Y[0].argmax(dim=1).item()))
    return ''.join([idx_to_char[i] for i in output])

In [21]:
predict_rnn('分开', 10, rnn, params, init_rnn_state, num_hiddens, vocab_size,
            device, idx_to_char, char_to_idx)

'分开花脸昨值祷丁风距宇横'

## 裁剪梯度
循环神经网络中较容易出现梯度衰减或梯度爆炸，我们可以裁剪梯度（clip gradient）。假设我们把所有模型参数梯度的元素拼接成一个向量 g，并设裁剪的阈值是θ。
裁剪后为$ \min(\frac{\theta}{||\boldsymbol{g}||},1) \boldsymbol{g}，$

In [22]:
def grad_clipping(params, theta, device):
    norm = torch.tensor([0.0], device=device)
    for param in params:
        norm += (param.grad.data ** 2).sum()
    norm = norm.sqrt().item()
    if norm > theta:
        for param in params:
            param.grad.data *= (theta / norm)

## 困惑度
我们通常使用困惑度（perplexity）来评价语言模型的好坏。回忆一下3.4节（softmax回归）中交叉熵损失函数的定义。困惑度是对交叉熵损失函数做指数运算后得到的值。特别地，

+ 最佳情况下，模型总是把标签类别的概率预测为1，此时困惑度为1； 
+ 最坏情况下，模型总是把标签类别的概率预测为0，此时困惑度为正无穷； 
+ 基线情况下，模型总是预测所有类别的概率都相同，此时困惑度为类别个数。 

显然，任何一个有效模型的困惑度必须小于类别个数。在本例中，困惑度必须小于词典大小vocab_size。

## 定义模型训练函数
这里的模型训练函数有以下几点不同：

1) 使用困惑度评价模型。 

2) 在迭代模型参数前裁剪梯度。

3) 对时序数据采用不同采样方法将导致隐藏状态初始化的不同。

In [23]:
def train_and_predict_rnn(rnn, get_params, init_rnn_state, num_hiddens,
                         vocab_size, device, corpus_indices, idx_to_char,
                         char_to_idx, is_random_iter, num_epochs, num_steps,
                         lr, clipping_theta, batch_size, pred_period,
                         pred_len, prefixes):
    if is_random_iter:
        data_iter_fn = data_iter_random
    else:
        data_iter_fn = data_iter_consecutive
    params = get_params()
    loss = nn.CrossEntropyLoss()
    for epoch in range(num_epochs):
        if not is_random_iter: #使用随机采样，每次初始化隐藏状态
            state = init_rnn_state(batch_size, num_hiddens, device)
        l_sum, n, start = 0.0, 0, time.time()
        data_iter = data_iter_fn(corpus_indices, batch_size, num_steps, device)
        for X, Y in data_iter:
            if is_random_iter:
                state = init_rnn_state(batch_size, num_hiddens, device) 
            else:
            # 相邻采样需要使用detach函数从计算图分离隐藏状态, 这是为了
            # 使模型参数的梯度计算只依赖一次迭代读取的小批量序列(防止梯度计算开销太大)
                for s in state:
                    s.detach()
            inputs = to_onehot(X, vocab_size)
            (outputs, state) = rnn(inputs, state, params)
            outputs = torch.cat(outputs, dim=0)
            y = torch.transpose(Y, 0, 1).contiguous().view(-1)
            l = loss(outputs, y.long())
            
            if params[0].grad is not None:
                for param in params:
                    param.grad.data.zero_()
            l.backward()
            grad_clipping(params, clipping_theta, device)
            sgd(params, lr, 1)
            l_sum += l.item() * y.shape[0]
            n += y.shape[0]
        if (epoch + 1) % pred_period == 0:
            print('epoch %d, perplexity %f, time %.2f sec' % (
                epoch + 1, math.exp(l_sum / n), time.time() - start))
            for prefix in prefixes:
                print(' -', predict_rnn(prefix, pred_len, rnn, params, init_rnn_state,
                    num_hiddens, vocab_size, device, idx_to_char, char_to_idx))

## 训练模型并创作歌词

In [24]:
num_epochs, num_steps, batch_size, lr, clipping_theta = 250, 35, 32, 1e2, 1e-2
pred_period, pred_len, prefixes = 50, 50, ['分开', '不分开']

In [25]:
train_and_predict_rnn(rnn, get_params, init_rnn_state, num_hiddens,
                      vocab_size, device, corpus_indices, idx_to_char,
                      char_to_idx, True, num_epochs, num_steps, lr,
                      clipping_theta, batch_size, pred_period, pred_len,
                      prefixes)

epoch 50, perplexity 69.576531, time 0.15 sec
 - 分开 我想要你想你 想知 你想你这 我想想你 你小了你 你的让我 你的让我 你的可爱女人  坏的让我疯狂
 - 不分开 我想想你想你 想知 我想你这想你 不知你的你不 我 想你你你 你的让我 我的可我 你的可爱女人  
epoch 100, perplexity 9.622164, time 0.14 sec
 - 分开 一直的美栈人 连有哈我的朋据 我想店的爱写有西元前 深埋在美索不达米亚 原真中 融始共 一九四有 
 - 不分开吗 我不要再想你 不知不觉 你已经离开我 不知不觉 我已经这节我 不知不觉 你已经离不我 不知不觉 
epoch 150, perplexity 2.916980, time 0.15 sec
 - 分开 快使用双 在小村外 恨给的梦  我的从界 我听多烦 你你在空 我的放空 恨我不外 说你没用 我的没
 - 不分开简 我的你以多 几色的美丽 谁分在美 相小了空屋 白慢蜡烛 温暖了日屋 白色蜡烛 温暖了日屋 白色蜡
epoch 200, perplexity 1.580704, time 0.16 sec
 - 分开 快使用双截过 哼哼哈兮 快使用双截棍 哼哼哈兮 习武之人切记 仁者无敌 是谁在练太极 风生水起 快
 - 不分开扫把的胖女巫 用拉丁文念咒语啦啦呜 她养的黑猫笑起来像哭 啦啦啦呜 一颗我怀手为猜透 心必让一个人慢
epoch 250, perplexity 1.312417, time 0.14 sec
 - 分开 一使用双拳 还一拳在身边的让尾蛇 无力像猫的狗 印地安老斑鸠 平常话不多 除非是乌鸦抢了它的窝 它
 - 不分开期 然后将过去 慢慢温习 让我爱上你 那场悲剧 是你完美演出的一场戏 宁愿心碎哭泣 再狠狠忘记 你爱


# 循环网络的简洁实现

## 定义模型
rnn_layer作为nn.RNN实例，在前向计算后会分别返回输出和隐藏状态h，其中输出指的是隐藏层在各个时间步上计算并输出的隐藏状态，它们通常作为后续输出层的输入。

In [26]:
num_hiddens = 256
rnn_layer = nn.RNN(input_size=vocab_size, hidden_size=num_hiddens)

In [27]:
num_steps = 35
batch_size = 2
state = None
X = torch.rand(num_steps, batch_size, vocab_size)
Y, state_new = rnn_layer(X, state)
print(Y.shape, len(state_new), state_new[0].shape)

torch.Size([35, 2, 256]) 1 torch.Size([2, 256])


In [28]:
class RNNModel(nn.Module):
    def __init__(self, rnn_layer, vocab_size):
        super(RNNModel, self).__init__()
        self.rnn = rnn_layer
        self.hidden_size = rnn_layer.hidden_size* (2 if rnn_layer.bidirectional else 1)
        self.vocab_size = vocab_size
        self.dense = nn.Linear(self.hidden_size, vocab_size)
        self.state = None
    
    def forward(self, inputs, state):
        X = to_onehot(inputs, self.vocab_size)
        Y, self.state = self.rnn(torch.stack(X), state)
        output = self.dense(Y.view(-1, Y.shape[-1]))
        return output, self.state

## 训练模型

In [29]:
def predict_rnn_pytorch(prefix, num_chars, model, vocab_size, device, idx_to_char,
                        char_to_idx):
    state = None
    #output会记录prefix加上输出
    output = [char_to_idx[prefix[0]]] 
    for t in range(num_chars + len(prefix)-1):
        X = torch.tensor([output[-1]], device=device).view(1,1)
        if state is not None:
            if isinstance(state, tuple):
                state = (state[0].to(device), state[1].to(device))
            else:
                state = state.to(device)
        (Y, state) = model(X, state)
        if t < len(prefix) -1:
            output.append(char_to_idx[prefix[t+1]])
        else:
            output.append(int(Y.argmax(dim=1).item()))
    return ''.join([idx_to_char[i] for i in output])

In [30]:
model = RNNModel(rnn_layer, vocab_size).to(device)
predict_rnn_pytorch('分开', 10, model, vocab_size, device, idx_to_char, char_to_idx)

'分开公撒撒撒撒撒撒撒撒撒'

In [31]:
#训练模型
def train_and_predict_rnn_pytorch(model, num_hiddens, vocab_size, device,
                                corpus_indices, idx_to_char, char_to_idx,
                                num_epochs, num_steps, lr, clipping_theta,
                                batch_size, pred_period, pred_len, prefixes):
    loss = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    model.to(device)
    state = None
    for epoch in range(num_epochs):
        l_sum, n, start = 0.0, 0, time.time()
        data_iter = data_iter_consecutive(corpus_indices, batch_size, num_steps, device)
        for X, Y in data_iter:
            if state is not None:
                if isinstance(state, tuple):
                    state = (state[0].detach(), state[1].detach())
                else:
                    state = state.detach()
            (output,state) = model(X, state)
            
            y = torch.transpose(Y, 0, 1).contiguous().view(-1)
            l = loss(output, y.long())
            
            optimizer.zero_grad()
            l.backward()
            grad_clipping(model.parameters(), clipping_theta, device)
            optimizer.step()
            l_sum += l.item() * y.shape[0]
            n += y.shape[0]
        
        try:
            perplexity = math.exp(l_sum / n)
        except OverflowError:
            perplexity = float('inf')
        if (epoch + 1) % pred_period == 0:
            print('epoch %d, perplexity %f, time %.2f sec' % (
                epoch + 1, perplexity, time.time() - start))
            for prefix in prefixes:
                print(' -', predict_rnn_pytorch(
                    prefix, pred_len, model, vocab_size, device, idx_to_char,
                    char_to_idx))

In [ ]:
#上一节实验中一样的超参数（除了学习率）来训练模型
num_epochs, batch_size, lr, clipping_theta = 550, 32, 1e-3, 1e-2 # 注意这里的学习率设置
pred_period, pred_len, prefixes = 50, 50, ['分开', '不分开']
train_and_predict_rnn_pytorch(model, num_hiddens, vocab_size, device,
                            corpus_indices, idx_to_char, char_to_idx,
                            num_epochs, num_steps, lr, clipping_theta,
                            batch_size, pred_period, pred_len, prefixes)

epoch 50, perplexity 8.218603, time 0.07 sec
 - 分开始我面 娘子 娘子依人的江南等我 泪不休 语沉默 娘子我人在江南 我 能发抖 我给你的生活  想要你
 - 不分开 我不能再想 我不要再想 我不 我不 我不 我不能再想 我不能再不 我不 再不 我不能再爱 我不能再
epoch 100, perplexity 1.243455, time 0.07 sec
 - 分开始我呼妈习惯 我给了这样打我妈妈 不要再想你我不要再想 我不 我不 我不要再想你 爱情来的太快就像龙
 - 不分开 我有多的纵  试上文字刻下了的那里我感轻够的落肉 我说店小二 三两银够不够 景色入秋 漫天黄沙凉过
epoch 150, perplexity 1.061572, time 0.07 sec
 - 分开 我不能再想 我不要再想 我不知我  你爱啊  我想了你了它我不要再想你 我有多 难你 没有你 我有
 - 不分开  我不著空 一片在梦里 在感动的穿爱女人的坏坏的让我疯狂的可爱女人 坏坏的让我疯狂的可爱女人 漂亮
epoch 200, perplexity 1.033613, time 0.07 sec
 - 分开 我不能再想 我不知不觉 你已经离开我 不知不觉 我跟了这节奏 后知后觉 又过了一个秋 后知后觉 我
 - 不分开  太多的空 一句的事 在回忆  牵在你在人 漂的还爱女人 坏坏的让我疯狂的可爱女人 坏坏的让我疯狂
epoch 250, perplexity 1.022081, time 0.08 sec
 - 分开 我不能再平 我不能 想要阻却陪我不的牛肉 我说店小二 三两银够不够 景色入秋 漫天黄沙凉过 塞北的
 - 不分开  又不放开  能知道觉 我该好好生活 我想要和说你眼睛知道我 是可爱让我不能的可爱你人 坏坏的让我
epoch 300, perplexity 1.015432, time 0.07 sec
 - 分开 我不知不觉 我已经离开我 不知不觉 我跟了这节奏 后知后觉 又过了一个秋 后知后觉 我该好好生活 
 - 不分开  又不放开 我有一可爱的我妈妈说原来我只想你我 陪我你看棒球 想这样没担忧 唱着歌 一直走 我想就
epoch 350, perplexity 1.019225, time 0.07 sec
 

# 门控循环单元（GRU）
它引入了重置门（reset gate）和更新门（update gate）的概念，从而修改了循环神经网络中隐藏状态的计算方式
门控循环神经网络可以更好地捕捉时间序列中时间步距离较大的依赖关系。
门控循环单元引入了门的概念，从而修改了循环神经网络中隐藏状态的计算方式。它包括重置门、更新门、候选隐藏状态和隐藏状态。  
重置门有助于捕捉时间序列里短期的依赖关系。  
更新门有助于捕捉时间序列里长期的依赖关系。  

## 读取数据集

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

(corpus_indices, char_to_idx, idx_to_char, vocab_size) = load_data_jay_lyrics()

## 从零开始实现

In [ ]:
num_inputs, num_hiddens, num_outputs = vocab_size, 256, vocab_size
print('will use',device)

def get_params():
    def _one(shape):
        ts = torch.tensor(np.random.normal(0, 0.01, size=shape), device=device, dtype=torch.float32)
        return torch.nn.Parameter(ts, requires_grad=True)
    def _three():
        return (_one((num_inputs, num_hiddens)),
                _one((num_hiddens, num_hiddens)),
                torch.nn.Parameter(torch.zeros(num_hiddens, device=device, dtype=torch.float32), requires_grad=True))

    W_xz, W_hz, b_z = _three()  # 更新门参数
    W_xr, W_hr, b_r = _three()  # 重置门参数
    W_xh, W_hh, b_h = _three()  # 候选隐藏状态参数

    # 输出层参数
    W_hq = _one((num_hiddens, num_outputs))
    b_q = torch.nn.Parameter(torch.zeros(num_outputs, device=device, dtype=torch.float32), requires_grad=True)
    return nn.ParameterList([W_xz, W_hz, b_z, W_xr, W_hr, b_r, W_xh, W_hh, b_h, W_hq, b_q])

### 定义模型

In [ ]:
def init_gru_state(batch_size, num_hiddens, device):
    return (torch.zeros((batch_size, num_hiddens), device=device))

In [ ]:
def gru(inputs, state, params):
    W_xz, W_hz, b_z, W_xr, W_hr, b_r, W_xh, W_hh, b_h, W_hq, b_q = params
    H = state
    outputs = []
    for X in inputs:
        Z = torch.sigmoid(torch.mm(X, W_xz) + torch.mm(H, W_hz) + b_z)
        R = torch.sigmoid(torch.mm(X, W_xr) + torch.mm(H, W_hr) + b_r)
        H_tilda = torch.tanh(torch.matmul(X, W_xh) + torch.matmul(R * H, W_hh) + b_h)
        H = Z * H + (1 - Z) * H_tilda
        Y = torch.matmul(H, W_hq) + b_q
        outputs.append(Y)
    return outputs, (H,)


In [ ]:
num_epochs, num_steps, batch_size, lr, clipping_theta = 160, 35, 32, 1e2, 1e-2
pred_period, pred_len, prefixes = 40, 50, ['分开', '不分开']

In [ ]:
train_and_predict_rnn(gru, get_params, init_gru_state, num_hiddens,
                          vocab_size, device, corpus_indices, idx_to_char,
                          char_to_idx, False, num_epochs, num_steps, lr,
                          clipping_theta, batch_size, pred_period, pred_len,
                          prefixes)

## 简洁实现

In [ ]:
lr = 1e-2 # 注意调整学习率
gru_layer = nn.GRU(input_size=vocab_size, hidden_size=num_hiddens)
model = RNNModel(gru_layer, vocab_size).to(device)
train_and_predict_rnn_pytorch(model, num_hiddens, vocab_size, device,
                                corpus_indices, idx_to_char, char_to_idx,
                                num_epochs, num_steps, lr, clipping_theta,
                                batch_size, pred_period, pred_len, prefixes)
